In [367]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By

In [368]:
def clean_data(file_path):
    data = []
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader, None)
        for row in reader:
            linkedin_url = row[1]
            if 'linkedin.com/in/' in linkedin_url:
                if not linkedin_url.startswith('https://'):
                    
                    row[1] = 'https://' + row[1][row[1].index('linkedin.com/in/'):]
                data.append(row)
    return data

In [369]:
def login_to_linkedin(driver, username, password):
    username_element = driver.find_element(By.NAME, "session_key")
    username_element.send_keys(username)
    password_element = driver.find_element(By.NAME, "session_password")
    password_element.send_keys(password)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()

In [378]:
def send_connection_requests(driver, data):
    failed = []
    success = []
    count = 1;
    for person in data:
        name = person[0]
        linkedin_url = person[1]
        print('Trying for ' + str(count) + ': ' + name)
        count += 1
        try:
            driver.get(linkedin_url)
            driver.implicitly_wait(5)
            name = driver.find_element(By.TAG_NAME, "h1").text
            print('Found: ' + name)
            try:
                blue_button = driver.find_element(By.CLASS_NAME, "artdeco-button--primary")
                button_html = blue_button.get_attribute('innerHTML').lower()
                if 'connect' in button_html:
                    driver.execute_script("arguments[0].click();", blue_button)
                    driver.implicitly_wait(1)
                    modal = driver.find_element(By.CLASS_NAME, 'artdeco-modal')
                    send_button = modal.find_element(By.CLASS_NAME, "artdeco-button--primary")
                    if 'Connect' in send_button.get_attribute('innerHTML'):
                        option = driver.find_element(By.CLASS_NAME, 'artdeco-pill--slate')
                        print(option.get_attribute('innerHTML'))
                        driver.execute_script("arguments[0].click();", option)
                        driver.implicitly_wait(1)
                        driver.execute_script("arguments[0].click();", send_button)
                        driver.implicitly_wait(1)
                    send_button = driver.find_element(By.CLASS_NAME, "artdeco-button--primary")
                    driver.execute_script("arguments[0].click();", send_button)
                    print('Connection Request Sent!')
                elif 'accept' in button_html:
        #             driver.execute_script("arguments[0].click();", blue_button)
                    print('Connection Request Accepted!')
                elif 'follow' in button_html:
                    print('Failed for Weird Case :/')
                    failed.append(person)
                elif 'message' in button_html or 'pending' in button_html:
                    print('Already Connected!')
                elif 'open' in button_html:
                    print('Yourself :)')
                else:
                    print('Failed for unknown reason :(')
                    failed.append(person)
            except:
                print('Already Connected!')
        except:
            print('Failed to connect to URL: ' + [linkedin_url[30]])
            failed.append(person)
        print()
    return failed

In [ ]:
USERNAME = ''
PASSWORD = ''

driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
driver.implicitly_wait(5)
login_to_linkedin(driver, USERNAME, PASSWORD)
driver.implicitly_wait(5)
file_path = './data.csv'
linkedin_links = clean_data(file_path)
failed = send_connection_requests(driver, linkedin_links)